In [ ]:
 # Dialer Accounts Early Cllection

# Dialer Accounts Early Cllection

In [ ]:
WITH ranked_data AS (
    SELECT 
        c.id AS client_id,
        l.id AS loan_id,
        l.issued_date,
        l.due_date,
        c.first_name,
        c.last_name,
        c.Email,
        c.National_ID,
        LPAD(c.mobile_phone, LENGTH(c.mobile_phone) + 1, '0') AS mobile_phone,  -- Adding leading zero to mobile phone
        EXTRACT(YEAR FROM AGE(CURRENT_DATE, c.Date_of_birth)) AS age,
        l.dpd,
        CASE
            WHEN l.derived_paid_total > 0 THEN 'Partially Paid'  -- If derived_paid_total > 0, set to 'Partially Paid'
            WHEN l.derived_paid_total = 0 THEN 'Not Paid'        -- If derived_paid_total = 0, set to 'Not Paid'
            ELSE 'Unknown'  -- Optional fallback for other values
        END AS Payment_Status,                     -- Changed 'derived_paid_total' to 'Payment Status'
        CASE
            WHEN l.installment_count = 1 THEN 'repeat'
            WHEN l.installment_count = 2 THEN 'new'
            ELSE 'unknown'  -- Optional fallback for other values
        END AS Loan_type,                     -- Changed 'installment_count' to 'Loan_type'
        l.loan_queue,
        ah.custom_data_json::json->>'activityType' AS activitytype,
        ah.custom_data_json::json->>'activityTopic' AS activitytopic,
        ah.custom_data_json::json->>'activityResolution' AS activityresolution,
        ah.custom_data_json::json->>'comment' AS comment,
        l.derived_ptp_status AS Current_PTP_Status,  -- Renaming derived_ptp_status as Current_PTP_Status using alias
        l.derived_outstanding_total,
        l.derived_principal_disbursed,
        l.derived_paid_total,-- Including derived_outstanding_total from loan table
        ROW_NUMBER() OVER (PARTITION BY c.id ORDER BY l.dpd ASC, ah.custom_data_json DESC) AS rn  -- Ranking rows per client_id
    FROM
        platform.client c
    LEFT JOIN
        platform.loan l ON c.id = l.client_id
        AND l.loan_status = 'ACTIVE'
        AND l.loan_queue IN ('EARLY_COLLECTIONS', 'LATE_COLLECTIONS', 'DEFAULTED')
    LEFT JOIN
        platform.activity_history ah ON c.id = ah.client_id
    WHERE
        l.client_id IS NOT NULL
        AND Dpd > 0
)
SELECT
    client_id,
    loan_id,
    issued_date,
    due_date,
    INITCAP(first_name) AS first_name,
    INITCAP(last_name) AS last_name,
    mobile_phone,
    Email,
    National_ID,
    age,
    dpd,
    Payment_Status,
    Loan_type,
    derived_principal_disbursed,
     derived_paid_total,
     derived_outstanding_total
FROM
    ranked_data
WHERE
    rn = 1  -- Select the first row for each client_id (removing duplicates)
ORDER BY
    dpd ASC, activitytype DESC;  -- Final ordering of results


# Ptp Effeciency Report

In [ ]:
WITH initial_data AS (
    -- Aggregate data for each user and promise_date
    SELECT
        lp.created_at,
        lp.updated_at,
        lp.created_by,
        u.user_name,
        lp.loan_id,
        lp.promise_status,
        lp.promise_amount,
        lp.promise_date,
        lp.due_date,
        CASE 
            -- Get the payment date for incoming payments between promise_date and due_date
            WHEN MAX(CASE WHEN pa.payment_type = 'INCOMING' 
                    AND pa.value_date BETWEEN lp.promise_date AND lp.due_date
                    AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID') 
                THEN pa.value_date ELSE NULL END) IS NOT NULL
            THEN MAX(CASE WHEN pa.payment_type = 'INCOMING' 
                    AND pa.value_date BETWEEN lp.promise_date AND lp.due_date
                    AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID') 
                THEN pa.value_date ELSE NULL END)
            ELSE NULL 
        END AS payment_date,
        CASE 
            -- Calculate amount paid based on conditions
            WHEN SUM(CASE WHEN pa.payment_type = 'INCOMING' 
                    AND pa.value_date BETWEEN lp.promise_date AND lp.due_date
                    AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID') 
                THEN CASE WHEN pa.amount > lp.promise_amount THEN lp.promise_amount ELSE pa.amount END ELSE 0 END) > lp.promise_amount
            THEN lp.promise_amount
            ELSE SUM(CASE WHEN pa.payment_type = 'INCOMING' 
                    AND pa.value_date BETWEEN lp.promise_date AND lp.due_date
                    AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID') 
                THEN CASE WHEN pa.amount > lp.promise_amount THEN lp.promise_amount ELSE pa.amount END ELSE 0 END)
        END AS amount_paid
    FROM 
        platform.loan_promise_to_pay lp
        LEFT JOIN platform.payment_allocation pa ON lp.loan_id = pa.loan_id
        LEFT JOIN platform.bo_user u ON lp.created_by = u.id
    WHERE 
        u.user_name IN ('andrew.kibet', 'benadette.omulo', 'caroline.mutuse', 'judy.kinyua', 'sammy.chege')
        AND lp.promise_date >= COALESCE(:promise_date_from, lp.promise_date)  -- Default to all promise_dates if :promise_date_from is not provided
        AND lp.promise_date <= COALESCE(:promise_date_to, lp.promise_date)    -- Default to all promise_dates if :promise_date_to is not provided
    GROUP BY 
        lp.created_at, lp.updated_at, lp.created_by, u.user_name, lp.loan_id, 
        lp.promise_status, lp.promise_amount, lp.promise_date, lp.due_date
),
activity_data AS (
    -- Aggregate activity data
    SELECT 
        ah.created_at::DATE AS promise_date,
        bo_user.user_name,
        COUNT(CASE WHEN ah.custom_data_json::json->>'activityResolution' IN ('COMPLETE', 'DC_COMPLETE_PTP', 'DC_REFUSAL') THEN 1 END) AS completed_calls,  -- Completed calls per day
        ROUND(
            COUNT(CASE WHEN ah.custom_data_json::json->>'activityResolution' IN ('COMPLETE', 'DC_COMPLETE_PTP', 'DC_REFUSAL') THEN 1 END)::NUMERIC 
            / NULLIF(COUNT(*), 0), 3
        ) AS completion_rate  -- Completion rate (completed calls / total calls)
    FROM platform.activity_history ah
    LEFT JOIN platform.bo_user ON ah.created_by = bo_user.id
    GROUP BY 
        ah.created_at::DATE, bo_user.user_name
)

-- Main query: Join both activity data and promise data
SELECT 
    subquery.user_name,
    SUM(subquery.total_ptp) AS total_ptp,
    SUM(subquery.ptp_paid) AS ptp_paid,
    ROUND(AVG(subquery.ptp_efficiency_by_count), 3) AS ptp_efficiency_by_count,

    SUM(subquery.total_ptp_amount) AS total_ptp_amount,
    SUM(subquery.ptp_amount_paid) AS ptp_amount_paid,
    ROUND(AVG(subquery.ptp_efficiency_by_amount), 3) AS ptp_efficiency_by_amount,

    SUM(subquery.active_ptp) AS active_ptp,
    SUM(subquery.active_ptp_amount) AS active_ptp_amount,
    ROUND(AVG(subquery.active_ptp_coverage), 3) AS active_ptp_coverage

FROM (
    -- Encapsulating the previous query
    SELECT 
        id.promise_date, 
        id.user_name,

        -- Loan promise data
        COUNT(CASE WHEN id.promise_status != 'CANCELLED' THEN id.loan_id END) AS total_ptp,
        COUNT(CASE WHEN id.promise_status = 'PAID' THEN id.loan_id END) AS ptp_paid,
        
        CASE 
            WHEN COUNT(CASE WHEN id.promise_status != 'CANCELLED' THEN id.loan_id END) > 0
            THEN ROUND(COUNT(CASE WHEN id.promise_status = 'PAID' THEN id.loan_id END) * 1.0 / COUNT(CASE WHEN id.promise_status != 'CANCELLED' THEN id.loan_id END), 3)
            ELSE 0
        END AS ptp_efficiency_by_count,

        SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END) AS total_ptp_amount,
        
        SUM(CASE WHEN id.promise_status IN ('PAID', 'PARTIALLY_PAID') THEN id.amount_paid ELSE 0 END) AS ptp_amount_paid,

        CASE 
            WHEN SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END) > 0
            THEN ROUND(SUM(CASE WHEN id.promise_status IN ('PAID', 'PARTIALLY_PAID') THEN id.amount_paid ELSE 0 END) * 1.0 / SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END), 3)
            ELSE 0
        END AS ptp_efficiency_by_amount,

        COUNT(CASE WHEN id.promise_status = 'ACTIVE' THEN id.loan_id END) AS active_ptp,
        SUM(CASE WHEN id.promise_status = 'ACTIVE' THEN id.promise_amount ELSE 0 END) AS active_ptp_amount,
        
        CASE 
            WHEN SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END) > 0
            THEN ROUND(SUM(CASE WHEN id.promise_status = 'ACTIVE' THEN id.promise_amount ELSE 0 END) * 1.0 / SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END), 3)
            ELSE 0
        END AS active_ptp_coverage

    FROM 
        initial_data id
    JOIN
        activity_data ad
        ON id.user_name = ad.user_name
        AND id.promise_date = ad.promise_date
    WHERE
        id.user_name IN ('andrew.kibet', 'benadette.omulo', 'caroline.mutuse', 'judy.kinyua', 'sammy.chege')
    GROUP BY 
        id.promise_date, id.user_name
) AS subquery
GROUP BY 
    subquery.user_name

UNION ALL

SELECT
    'Total' AS user_name,  -- Add "Total" row
    SUM(total_ptp) AS total_ptp,
    SUM(ptp_paid) AS ptp_paid,
    ROUND(AVG(ptp_efficiency_by_count), 3) AS ptp_efficiency_by_count,

    SUM(total_ptp_amount) AS total_ptp_amount,
    SUM(ptp_amount_paid) AS ptp_amount_paid,
    ROUND(AVG(ptp_efficiency_by_amount), 3) AS ptp_efficiency_by_amount,

    SUM(active_ptp) AS active_ptp,
    SUM(active_ptp_amount) AS active_ptp_amount,
    ROUND(AVG(active_ptp_coverage), 3) AS active_ptp_coverage

FROM (
    -- Reuse the subquery part of the main query to calculate totals
    SELECT 
        id.promise_date, 
        id.user_name,

        -- Loan promise data
        COUNT(CASE WHEN id.promise_status != 'CANCELLED' THEN id.loan_id END) AS total_ptp,
        COUNT(CASE WHEN id.promise_status = 'PAID' THEN id.loan_id END) AS ptp_paid,
        
        CASE 
            WHEN COUNT(CASE WHEN id.promise_status != 'CANCELLED' THEN id.loan_id END) > 0
            THEN ROUND(COUNT(CASE WHEN id.promise_status = 'PAID' THEN id.loan_id END) * 1.0 / COUNT(CASE WHEN id.promise_status != 'CANCELLED' THEN id.loan_id END), 3)
            ELSE 0
        END AS ptp_efficiency_by_count,

        SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END) AS total_ptp_amount,
        
        SUM(CASE WHEN id.promise_status IN ('PAID', 'PARTIALLY_PAID') THEN id.amount_paid ELSE 0 END) AS ptp_amount_paid,

        CASE 
            WHEN SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END) > 0
            THEN ROUND(SUM(CASE WHEN id.promise_status IN ('PAID', 'PARTIALLY_PAID') THEN id.amount_paid ELSE 0 END) * 1.0 / SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END), 3)
            ELSE 0
        END AS ptp_efficiency_by_amount,

        COUNT(CASE WHEN id.promise_status = 'ACTIVE' THEN id.loan_id END) AS active_ptp,
        SUM(CASE WHEN id.promise_status = 'ACTIVE' THEN id.promise_amount ELSE 0 END) AS active_ptp_amount,
        
        CASE 
            WHEN SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END) > 0
            THEN ROUND(SUM(CASE WHEN id.promise_status = 'ACTIVE' THEN id.promise_amount ELSE 0 END) * 1.0 / SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END), 3)
            ELSE 0
        END AS active_ptp_coverage

    FROM 
        initial_data id
    JOIN
        activity_data ad
        ON id.user_name = ad.user_name
        AND id.promise_date = ad.promise_date
    WHERE
        id.user_name IN ('andrew.kibet', 'benadette.omulo', 'caroline.mutuse', 'judy.kinyua', 'sammy.chege')
    GROUP BY 
        id.promise_date, id.user_name
) AS subquery;


# Cash Flow

In [ ]:
SELECT 
    pa.value_date,  
    -- Total Disbursements (OUTGOING with 'Disbursement/Settle')
    SUM(CASE 
            WHEN pa.payment_type = 'OUTGOING' AND pa.tx_type IN ('Disbursement/Settle') AND pa.Reversed = false THEN pa.amount 
            ELSE 0 
        END) AS Disbursements,  

    -- Total Charges (OUTGOING with 'Other')
    SUM(CASE 
            WHEN pa.payment_type = 'OUTGOING' AND pa.tx_type IN ('Other') AND pa.Reversed = false THEN pa.amount 
            ELSE 0 
        END) AS Charges, 

    -- Total Outflow (All OUTGOING payments)
    SUM(CASE 
            WHEN pa.payment_type = 'OUTGOING' AND pa.Reversed = false THEN pa.amount 
            ELSE 0 
        END) AS Total_outflow,  

    -- Total Inflow (INCOMING with 'Loan/Repay')
    SUM(CASE 
            WHEN pa.payment_type = 'INCOMING' AND pa.tx_type IN ('Loan/Repay') AND pa.Reversed = false THEN pa.amount 
            ELSE 0 
        END) AS Total_inflow 

FROM 
    platform.payment_allocation pa
WHERE
    pa.value_date >= COALESCE(:Value_date_from, pa.value_date)  -- Default to no filter if :Value_date_from is NULL
    AND pa.value_date <= COALESCE(:Value_date_to, pa.value_date)  -- Default to no filter if :Value_date_to is NULL
    AND pa.Reversed = false  -- Ensure only records with Reversed set to false are considered

GROUP BY 
    pa.value_date  -- Group by value_date for daily totals
ORDER BY 
    pa.value_date ASC;  -- Order by value_date ascending


# Disbursements Vs Repayments

In [ ]:
WITH loan_data AS (
    -- Code 1: Logic to select loan details
    SELECT DISTINCT ON (id) 
        id, 
        client_id, 
        loan_status, 
        issued_date,
        issued_at,
        CASE 
            WHEN loan_sequence_nr = 1 THEN 'New' 
            ELSE 'Repeated' 
        END AS client_type,
        CASE 
            WHEN loan_status = 'PAID' THEN 'Paid'
            WHEN loan_status = 'ACTIVE' AND COALESCE(derived_paid_total, 0) = 0 THEN 'Not_paid'
            WHEN loan_status = 'ACTIVE' AND COALESCE(derived_paid_total, 0) > 0 THEN 'Partially_paid'
            ELSE 'Other' 
        END AS payment_status,
        closed_date, 
        original_closed_at, 
        EXTRACT(HOUR FROM original_closed_at) AS closed_hour,
        TRIM(TO_CHAR(original_closed_at, 'Day')) AS closed_day_of_week,
        due_date, 
        derived_principal_disbursed, 
        derived_charged_total, 
        derived_outstanding_total, 
        COALESCE(derived_paid_total, 0) AS derived_paid_total,
        derived_past_due_total
    FROM 
        platform.loan
    ORDER BY 
        id, issued_date DESC
),
ptp_aggregated AS (
    -- Code 2: Logic to calculate payment amounts
    SELECT sub.loan_id, 
           SUM(sub.amount_paid) AS Ptp_amount
    FROM (
        SELECT 
            lp.loan_id,
            CASE 
                WHEN SUM(CASE 
                            WHEN pa.payment_type = 'INCOMING' 
                                 AND pa.value_date BETWEEN lp.promise_date AND lp.due_date
                                 AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID')
                            THEN 
                                CASE 
                                    WHEN pa.amount > lp.promise_amount THEN lp.promise_amount  -- Cap the amount to promise_amount
                                    ELSE pa.amount 
                                END
                            ELSE 0 
                        END) > lp.promise_amount THEN lp.promise_amount
                ELSE SUM(CASE 
                            WHEN pa.payment_type = 'INCOMING' 
                                 AND pa.value_date BETWEEN lp.promise_date AND lp.due_date
                                 AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID')
                            THEN 
                                CASE 
                                    WHEN pa.amount > lp.promise_amount THEN lp.promise_amount  -- Cap the amount to promise_amount
                                    ELSE pa.amount 
                                END
                            ELSE 0 
                        END)
            END AS amount_paid
        FROM platform.loan_promise_to_pay lp
        LEFT JOIN platform.payment_allocation pa
            ON lp.loan_id = pa.loan_id
        WHERE 
            lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID')
        GROUP BY 
            lp.created_at,
            lp.updated_at,
            lp.created_by,
            lp.loan_id, 
            lp.promise_amount, 
            lp.promise_date, 
            lp.due_date
    ) sub
    GROUP BY sub.loan_id
)
-- Final SELECT to join loan data with aggregated payment data
SELECT 
    ld.id, 
    ld.client_id, 
    ld.loan_status, 
    ld.issued_date, 
    ld.issued_at,
    ld.payment_status,
    ld.client_type,
    ld.closed_date, 
    ld.original_closed_at, 
    ld.closed_hour, 
    ld.closed_day_of_week,
    ld.due_date, 
    ld.derived_principal_disbursed, 
    ld.derived_charged_total, 
    ld.derived_outstanding_total, 
    ld.derived_paid_total,
    ld.derived_past_due_total,
    pa.Ptp_amount  -- Add the aggregated payment amount from Code 2
FROM 
    loan_data ld
LEFT JOIN 
    ptp_aggregated pa ON ld.id = pa.loan_id
-- Apply filters using the correct format
WHERE 
    ld.issued_date BETWEEN COALESCE(:Issued_date_from, ld.issued_date) 
    AND COALESCE(:Issued_date_to, ld.issued_date)
    AND ld.due_date BETWEEN COALESCE(:due_date_from, ld.due_date) 
    AND COALESCE(:due_date_to, ld.due_date)
    AND ld.loan_status IN ('ACTIVE', 'PAID')
    AND ld.client_type = COALESCE(:client_type, ld.client_type)
ORDER BY 
    ld.issued_date;


# Expected vs Paid by Due Date

In [ ]:
WITH loan_data AS (
    -- Code 1: Logic to select loan details
    SELECT DISTINCT ON (id)
        id, 
        client_id, 
        loan_status, 
        issued_date, 
        issued_at, 
        CASE WHEN loan_sequence_nr = 1 THEN 'New' ELSE 'Repeated' END AS client_type,
        CASE 
            WHEN loan_status = 'PAID' THEN 'Paid'
            WHEN loan_status = 'ACTIVE' AND COALESCE(derived_paid_total, 0) = 0 THEN 'Not_paid'
            WHEN loan_status = 'ACTIVE' AND COALESCE(derived_paid_total, 0) > 0 THEN 'Partially_paid'
            ELSE 'Other'
        END AS payment_status,
        closed_date, 
        original_closed_at, 
        EXTRACT(HOUR FROM original_closed_at) AS closed_hour,
        TRIM(TO_CHAR(original_closed_at, 'Day')) AS closed_day_of_week,
        due_date, 
        derived_principal_disbursed, 
        derived_charged_total, 
        derived_outstanding_total, 
        COALESCE(derived_paid_total, 0) AS derived_paid_total, 
        derived_past_due_total
    FROM platform.loan
    ORDER BY id, issued_date DESC
),
ptp_aggregated AS (
    -- Code 2: Logic to calculate payment amounts
    SELECT 
        sub.loan_id, 
        SUM(sub.amount_paid) AS Ptp_amount
    FROM (
        SELECT 
            lp.loan_id,
            CASE 
                WHEN SUM(CASE WHEN pa.payment_type = 'INCOMING' 
                    AND pa.value_date BETWEEN lp.promise_date AND lp.due_date 
                    AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID')
                    THEN 
                        CASE WHEN pa.amount > lp.promise_amount THEN lp.promise_amount 
                        ELSE pa.amount END
                    ELSE 0 END) > lp.promise_amount THEN lp.promise_amount
                ELSE SUM(CASE WHEN pa.payment_type = 'INCOMING' 
                    AND pa.value_date BETWEEN lp.promise_date AND lp.due_date 
                    AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID')
                    THEN 
                        CASE WHEN pa.amount > lp.promise_amount THEN lp.promise_amount 
                        ELSE pa.amount END
                    ELSE 0 END)
            END AS amount_paid
        FROM platform.loan_promise_to_pay lp
        LEFT JOIN platform.payment_allocation pa 
            ON lp.loan_id = pa.loan_id
        WHERE lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID')
        GROUP BY lp.created_at, lp.updated_at, lp.created_by, lp.loan_id, 
                 lp.promise_amount, lp.promise_date, lp.due_date
    ) sub
    GROUP BY sub.loan_id
)
-- Final SELECT to join loan data with aggregated payment data and SUM the totals by due_date
SELECT 
    ld.due_date, 
    SUM(ld.derived_charged_total) AS total_derived_charged, 
    SUM(ld.derived_paid_total) AS total_derived_paid, 
     SUM(ld.derived_outstanding_total) AS total_derived_outstanding, 
    SUM(pa.Ptp_amount) AS total_ptp_amount  -- Sum the payment amounts from Code 2
FROM loan_data ld
LEFT JOIN ptp_aggregated pa 
    ON ld.id = pa.loan_id -- Correct reference
WHERE ld.due_date BETWEEN COALESCE(:due_date_from, ld.due_date) 
    AND COALESCE(:due_date_to, ld.due_date)
GROUP BY ld.due_date -- Aggregate by due_date
ORDER BY ld.due_date;


# Collection Performance Report

In [ ]:
WITH initial_data AS (
    -- Aggregate data for each user and promise_date
    SELECT
        lp.created_at,
        lp.updated_at,
        lp.created_by,
        u.user_name,
        lp.loan_id,
        lp.promise_status,
        lp.promise_amount,
        lp.promise_date,
        lp.due_date,
        CASE 
            WHEN MAX(CASE WHEN pa.payment_type = 'INCOMING' 
                    AND pa.value_date BETWEEN lp.promise_date AND lp.due_date
                    AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID') 
                THEN pa.value_date ELSE NULL END) IS NOT NULL
            THEN MAX(CASE WHEN pa.payment_type = 'INCOMING' 
                    AND pa.value_date BETWEEN lp.promise_date AND lp.due_date
                    AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID') 
                THEN pa.value_date ELSE NULL END)
            ELSE NULL 
        END AS payment_date,
        CASE 
            WHEN SUM(CASE WHEN pa.payment_type = 'INCOMING' 
                    AND pa.value_date BETWEEN lp.promise_date AND lp.due_date
                    AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID') 
                THEN CASE WHEN pa.amount > lp.promise_amount THEN lp.promise_amount ELSE pa.amount END ELSE 0 END) > lp.promise_amount
            THEN lp.promise_amount
            ELSE SUM(CASE WHEN pa.payment_type = 'INCOMING' 
                    AND pa.value_date BETWEEN lp.promise_date AND lp.due_date
                    AND lp.promise_status IN ('ACTIVE', 'PARTIALLY_PAID', 'PAID') 
                THEN CASE WHEN pa.amount > lp.promise_amount THEN lp.promise_amount ELSE pa.amount END ELSE 0 END)
        END AS amount_paid
    FROM 
        platform.loan_promise_to_pay lp
        LEFT JOIN platform.payment_allocation pa ON lp.loan_id = pa.loan_id
        LEFT JOIN platform.bo_user u ON lp.created_by = u.id
    WHERE 
        u.user_name IN ('andrew.kibet', 'benadette.omulo', 'caroline.mutuse', 'judy.kinyua', 'sammy.chege')
    GROUP BY 
        lp.created_at, lp.updated_at, lp.created_by, u.user_name, lp.loan_id, 
        lp.promise_status, lp.promise_amount, lp.promise_date, lp.due_date
),
activity_data AS (
    -- Aggregate activity data
    SELECT 
        ah.created_at::DATE,
        bo_user.user_name,
        COUNT(
            CASE 
                WHEN ah.custom_data_json::json->>'activityResolution' IN (
                    'COMPLETE', 'SWITCHED_OFF', 'NOT_ANSWERED', 'HANG_UP',
                    'DC_COMPLETE_PTP', 'BUSY', 'DC_HANG_UP', 'VOICE_MAIL',
                    'DC_REFUSAL', 'WRONG_DATA', 'CC_SILENT_CALL', 'DC_HPF'
                ) THEN 1 
            END
        ) AS total_calls,
        COUNT(
            CASE 
                WHEN ah.custom_data_json::json->>'activityResolution' IN ('COMPLETE', 'DC_COMPLETE_PTP', 'DC_REFUSAL') 
                THEN 1 
            END
        ) AS completed_calls,
        ROUND(
            COUNT(
                CASE 
                    WHEN ah.custom_data_json::json->>'activityResolution' IN ('COMPLETE', 'DC_COMPLETE_PTP', 'DC_REFUSAL') 
                    THEN 1 
                END
            )::NUMERIC / NULLIF(
                COUNT(
                    CASE 
                        WHEN ah.custom_data_json::json->>'activityResolution' IN (
                            'COMPLETE', 'SWITCHED_OFF', 'NOT_ANSWERED', 'HANG_UP',
                            'DC_COMPLETE_PTP', 'BUSY', 'DC_HANG_UP', 'VOICE_MAIL',
                            'DC_REFUSAL', 'WRONG_DATA', 'CC_SILENT_CALL', 'DC_HPF'
                        ) THEN 1 
                    END
                ), 0
            ), 3
        ) AS completion_rate
    FROM 
        platform.activity_history ah
    LEFT JOIN 
        platform.bo_user ON ah.created_by = bo_user.id
    GROUP BY 
        ah.created_at::DATE, bo_user.user_name
),
Q AS (
    -- Main query: Join both activity data and promise data
    SELECT 
        subquery.user_name,
        
        -- Activity data first
        SUM(subquery.total_calls) AS total_calls,
        SUM(subquery.completed_calls) AS completed_calls,
        ROUND(AVG(subquery.completion_rate), 3) AS completion_rate,

        -- Loan promise data next
        SUM(subquery.total_ptp) AS total_ptp,
        
        -- New ptp_booking_rate calculation placed between total_ptp and ptp_paid
        ROUND(SUM(subquery.total_ptp) * 1.0 / NULLIF(SUM(subquery.completed_calls), 0), 3) AS ptp_booking_rate,

        SUM(subquery.ptp_paid) AS ptp_paid,

        -- ptp_efficiency_by_count comes immediately after ptp_paid
        ROUND(AVG(subquery.ptp_efficiency_by_count), 3) AS ptp_efficiency_by_count,
        
        SUM(subquery.total_ptp_amount) AS total_ptp_amount,
        SUM(subquery.ptp_amount_paid) AS ptp_amount_paid,

        -- ptp_efficiency_by_amount stays in its original position
        ROUND(AVG(subquery.ptp_efficiency_by_amount), 3) AS ptp_efficiency_by_amount,

        SUM(subquery.active_ptp) AS active_ptp,
        SUM(subquery.active_ptp_amount) AS active_ptp_amount,
        ROUND(AVG(subquery.active_ptp_coverage), 3) AS active_ptp_coverage

    FROM (
        -- Encapsulating the previous query
        SELECT 
            id.promise_date, 
            id.user_name,

            -- Activity data
            ad.total_calls,
            ad.completed_calls,
            ad.completion_rate,

            -- Loan promise data
            COUNT(CASE WHEN id.promise_status != 'CANCELLED' THEN id.loan_id END) AS total_ptp,
            COUNT(CASE WHEN id.promise_status = 'PAID' THEN id.loan_id END) AS ptp_paid,

            SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END) AS total_ptp_amount,
            SUM(CASE WHEN id.promise_status IN ('PAID', 'PARTIALLY_PAID') THEN id.amount_paid ELSE 0 END) AS ptp_amount_paid,

            CASE 
                WHEN SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END) > 0
                THEN ROUND(SUM(CASE WHEN id.promise_status IN ('PAID', 'PARTIALLY_PAID') THEN id.amount_paid ELSE 0 END) * 1.0 / SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END), 3)
                ELSE 0
            END AS ptp_efficiency_by_amount,

            -- Added ptp_efficiency_by_count calculation
            CASE 
                WHEN COUNT(CASE WHEN id.promise_status != 'CANCELLED' THEN id.loan_id END) > 0
                THEN ROUND(COUNT(CASE WHEN id.promise_status = 'PAID' THEN id.loan_id END) * 1.0 / COUNT(CASE WHEN id.promise_status != 'CANCELLED' THEN id.loan_id END), 3)
                ELSE 0
            END AS ptp_efficiency_by_count,

            COUNT(CASE WHEN id.promise_status = 'ACTIVE' THEN id.loan_id END) AS active_ptp,
            SUM(CASE WHEN id.promise_status = 'ACTIVE' THEN id.promise_amount ELSE 0 END) AS active_ptp_amount,
            
            CASE 
                WHEN SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END) > 0
                THEN ROUND(SUM(CASE WHEN id.promise_status = 'ACTIVE' THEN id.promise_amount ELSE 0 END) * 1.0 / SUM(CASE WHEN id.promise_status != 'CANCELLED' THEN id.promise_amount ELSE 0 END), 3)
                ELSE 0
            END AS active_ptp_coverage

        FROM 
            initial_data id
        JOIN
            activity_data ad
            ON id.user_name = ad.user_name
            AND id.promise_date = ad.created_at
        WHERE
            id.user_name IN ('andrew.kibet', 'benadette.omulo', 'caroline.mutuse', 'judy.kinyua', 'sammy.chege')
            AND id.promise_date >= COALESCE(:promise_date_from, id.promise_date)  -- Apply promise_date filter here
            AND id.promise_date <= COALESCE(:promise_date_to, id.promise_date)    -- Apply promise_date filter here
        GROUP BY 
            id.promise_date, id.user_name, ad.total_calls, ad.completed_calls, ad.completion_rate
    ) AS subquery
    GROUP BY 
        subquery.user_name
)

-- Final query: Select the data for each user and then the total row
SELECT 
    user_name,
    total_calls,
    completed_calls,
    completion_rate,
    total_ptp,
    ptp_booking_rate,
    ptp_paid,
    ptp_efficiency_by_count,  -- ptp_efficiency_by_count calculation added
    total_ptp_amount,
    ptp_amount_paid,
    ptp_efficiency_by_amount,
    active_ptp,
    active_ptp_amount,
    active_ptp_coverage
FROM Q

UNION ALL

-- Total row calculation
SELECT
    'Total' AS user_name,
    SUM(total_calls) AS total_calls,
    SUM(completed_calls) AS completed_calls,
    ROUND(AVG(completion_rate), 3) AS completion_rate,
    SUM(total_ptp) AS total_ptp,
    ROUND(SUM(total_ptp) * 1.0 / NULLIF(SUM(completed_calls), 0), 3) AS ptp_booking_rate,
    SUM(ptp_paid) AS ptp_paid,
    ROUND(AVG(ptp_efficiency_by_count), 3) AS ptp_efficiency_by_count,
    SUM(total_ptp_amount) AS total_ptp_amount,
    SUM(ptp_amount_paid) AS ptp_amount_paid,
    ROUND(AVG(ptp_efficiency_by_amount), 3) AS ptp_efficiency_by_amount,
    SUM(active_ptp) AS active_ptp,
    SUM(active_ptp_amount) AS active_ptp_amount,
    ROUND(AVG(active_ptp_coverage), 3) AS active_ptp_coverage
FROM Q;
